# Wine quality prediction

In [6]:
import logging
import warnings
import os

import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from skl2onnx import to_onnx

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

MLFLOW_PORT = os.getenv("MLFLOW_PORT", "8080")
mlflow.set_tracking_uri(f"http://localhost:{MLFLOW_PORT}")


In [1]:
import logging
import warnings
import os

import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from skl2onnx import to_onnx

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

MLFLOW_PORT = os.getenv("MLFLOW_PORT", "8080")
TRITON_PORT_GRPC = os.getenv("TRITON_PORT_GRPC", "8001")
ONNX_MODEL_NAME = "wine-quality-onnx"

mlflow.set_tracking_uri(f"http://localhost:{MLFLOW_PORT}")

# Read the wine-quality csv file from the URL
csv_url = (
    "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
)
try:
    data = pd.read_csv(csv_url, sep=";")
except Exception as e:
    logger.exception(
        "Unable to download training & test CSV, check your internet connection. Error: %s", e
    )

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]


## Step 1 - training

This tutorial is inspired from [here](https://www.mlflow.org/docs/1.20.2/tutorials-and-examples/tutorial.html). It showcases how you can use MLflow end-to-end to:
- Train a linear regression model
- Package the code that trains the model in a reusable and reproducible model format
- Deploy the model

This tutorial uses a dataset to predict the quality of wine based on quantitative features like the wine’s “fixed acidity”, “pH”, “residual sugar”, and so on.

The data set used in this example is from http://archive.ics.uci.edu/ml/datasets/Wine+Quality
P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.


In [ ]:
alpha = 0.1
l1_ratio = 0.1

def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

warnings.filterwarnings("ignore")
np.random.seed(40)

print("ready to start")

# Useful for multiple runs (only doing one run in this sample notebook)
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    print("starting run")
    # Execute ElasticNet
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    # Evaluate Metrics
    predicted_qualities = lr.predict(test_x)
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    # Print out metrics
    print(f"Elasticnet model (alpha={alpha:f}, l1_ratio={l1_ratio:f}):")
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    # Infer model signature
    predictions = lr.predict(train_x)
    signature = infer_signature(train_x, predictions)

    # Log parameter, metrics, and model to MLflow
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    mlflow.sklearn.log_model(lr, "wine-quality-sklearn", signature=signature)

    # convert to onnx
    onx = to_onnx(lr, train_x[:1].to_numpy())
    mlflow.onnx.log_model(onx, ONNX_MODEL_NAME, signature=signature, registered_model_name=ONNX_MODEL_NAME)


## Step 2 - run the model locally

We use the mlflow tracking server to download and run the model

In [ ]:
import mlflow.onnx
import onnxruntime as ort
from mlflow import MlflowClient

# retrieve the most recent model version
mlflow_client = MlflowClient()
model_version = max(m.version for m in mlflow_client.search_model_versions(f"name='{ONNX_MODEL_NAME}'"))

# load the model from mlflow
model = mlflow.onnx.load_model(f"models:/{ONNX_MODEL_NAME}/{model_version}")

# run the model with onnxruntime
onnx_session = ort.InferenceSession(model.SerializeToString())
input_name = onnx_session.get_inputs()[0].name
test_results = onnx_session.run(None, {input_name: np.array(test_x[:4])})
test_results

## Step 3 - update triton inference server with the new model

Note that the same operation could be executed with the cmdline : 

```
mlflow deployments create -t triton --flavor onnx --name wine-quality -m models:/wine-quality-onnx/<MODEL_VERSION>
```

In [ ]:
from mlflow import MlflowClient
from mlflow.deployments import get_deploy_client

# retrieve the most recent model version
mlflow_client = MlflowClient()
model_version = max(m.version for m in mlflow_client.search_model_versions(f"name='{ONNX_MODEL_NAME}'"))

client = get_deploy_client('triton')
client.create_deployment(ONNX_MODEL_NAME, f"models:/{ONNX_MODEL_NAME}/{model_version}", flavor="onnx")


## Step 4 - run inference from the model deployed in the triton inference server

In [ ]:
import tritonclient.grpc as grpcclient
triton_client = grpcclient.InferenceServerClient(
    url=f"localhost:{TRITON_PORT_GRPC}", verbose=True
)
input = grpcclient.InferInput("X", [4,11], "FP64")
input.set_data_from_numpy(np.atleast_2d(test_x.iloc[:4].to_numpy()))
result = triton_client.infer(ONNX_MODEL_NAME, inputs=[input])
result.as_numpy("variable")